In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import re
url = 'https://raw.githubusercontent.com/patriciof3/mentoria13_famaf_2024/main/df_reddit_mentoria.csv'
df = pd.read_csv(url)

# **Trabajo Práctico 2**

# Contenido

## Stemming y Lemmatization

Una vez que limpiamos adecuadamente el df, lo que debemos hacer es un proceso de normalización de palabras, es decir, reducir la variabilidad que existe entre vocablos de la misma familia.

Para ello existen en principio dos técnicas diferentes:

- Stemming: Cortar las palabras para dejar solo su raíz
- Lemmatización: Se utilizan librerías con funciones preentrenadas

In [29]:
# tomamos una entrada y removemos caracteres especiales
example = re.sub(r'[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ\s]', '', df["content"][130])
example

'Empecé con esto desde hace ya tiempo  años las razones son variadas ya que hay historial de enfermedades mentales en mi familia y pase por un momento duro yo sola en aquel entonces Pedi ayuda  años después y estuve medicada por un período muy corto de tiempo He sentido de todo desde las sensaciones más leves hasta las que tengo día con día En particular sufro de mareos y cuando estoy en la escuela tengo dificultad para respirar Mi punto es ustedes creen que haya alguien superado esto por completo Había aceptado vivir asi pero despues empecé a comparar mi vida con la de otras personas y quisiera al menos vivir sin tener esta mentalidad Hace no mucho tiempo logré controlarme Estaba a punto de explotar en una plaza comercial y simplemente no me lo permití Tuve una muy buena semana después de eso pero estar en calma es tan agotador  más porque nunca pensé que tuviera que esforzarme en ser feliz'

In [30]:
import nltk
from nltk.stem.snowball import SnowballStemmer
import spacy

nlp = spacy.load("es_core_news_sm")

# remover stopwords

stop_words = spacy.lang.es.stop_words.STOP_WORDS

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.text.lower() not in stop_words]
    return ' '.join(tokens)

example_stopwords = preprocess_text(example)

doc = nlp(example_stopwords)

tokenized = tokens = [token.text for token in doc]


# Stemming
stemmer = SnowballStemmer('spanish')

stemmed_words = [stemmer.stem(word) for word in tokenized]

# Lematización

lemmatized_words = [token.lemma_ for token in doc]

df_example = pd.DataFrame(
    {'Original': tokenized,
     'Stemming': stemmed_words,
     'Lemmatization': lemmatized_words
    })


In [31]:
df_filtered = df_example[df_example.nunique(axis=1) == df_example.shape[1]]
df_filtered

,Original,Stemming,Lemmatization
0,Empecé,empec,empezar
4,razones,razon,razón
5,variadas,vari,variado
10,pase,pas,pasar
14,ayuda,ayud,ayudar
17,estuve,estuv,estar
18,medicada,medic,medicado
23,sensaciones,sensacion,sensación
24,leves,lev,leve
32,creen,cre,creer


## Vectorización

Una vez que tenemos un df apropiadamente curado, podemos pasar a realizar la vectorización de nuestros datos. Bag of Words (BoW) y TF-IDF (Term Frequency-Inverse Document Frequency) son dos técnicas fundamentales utilizadas en el procesamiento de lenguaje natural (NLP) para la representación de textos.

### Bag of Words

Representa el texto como una bolsa de sus palabras, sin tener en cuenta el orden de las palabras ni su gramática. Cada documento se convierte en un vector de frecuencias de palabras, donde cada dimensión del vector corresponde a una palabra del vocabulario.

### TF-IDF
Pondera las palabras en un documento basado en su frecuencia en el documento (TF) y en la frecuencia inversa de documentos (IDF) que contienen esa palabra en el corpus. La idea es reducir la importancia de las palabras comunes y resaltar las palabras raras pero relevantes. Un valor alto de TF-IDF indica que el término es muy relevante para ese documento en particular y no es común en el corpus. Un valor bajo indica que el término no es muy relevante para ese documento específico, o es muy común en el corpus y, por lo tanto, no aporta mucha información.

In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
pd.set_option('display.max_columns', None)
pd.options.display.float_format = "{:,.2f}".format

# Crear DataFrame dummy con comentarios de Reddit
data = {
    'content': [
        'Me siento muy ansioso últimamente',
        'No puedo dormir debido a la ansiedad',
        'Me siento deprimido y sin energía',
        'La depresión está arruinando mi vida',
        'Estoy teniendo ataques de pánico frecuentemente',
        'La tristeza es constante y no sé qué hacer'
    ],
    'subreddit': ['Ansiedad', 'Ansiedad', 'Depresión', 'Depresión', 'Ansiedad', 'Depresión']
}

df_example = pd.DataFrame(data)

def preprocess(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Aplicar la función de preprocesamiento a la columna de contents
df_example['content'] = df_example['content'].apply(preprocess)

# Implementación de Bag of Words
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(df_example['content'])

# Convertir a DataFrame para mejor visualización
df_bow = pd.DataFrame(X_bow.toarray(), columns=vectorizer_bow.get_feature_names_out())

# Implementación de TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(df_example['content'])

# Convertir a DataFrame para mejor visualización
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

# Mostrar los DataFrames
print("Bag of Words (BoW):")
display(df_bow)

print("\nTF-IDF:")
display(df_tfidf)

Bag of Words (BoW):


,ansiedad,ansioso,arruinar,ataque,constante,depresión,deprimido,dormir,energía,frecuentemente,pánico,sentir,tener,tristeza,vida,últimamente
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0



TF-IDF:


,ansiedad,ansioso,arruinar,ataque,constante,depresión,deprimido,dormir,energía,frecuentemente,pánico,sentir,tener,tristeza,vida,últimamente
0,0.00,0.61,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.61
1,0.71,0.00,0.00,0.00,0.00,0.00,0.00,0.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.61,0.00,0.61,0.00,0.00,0.50,0.00,0.00,0.00,0.00
3,0.00,0.00,0.58,0.00,0.00,0.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.58,0.00
4,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.50,0.50,0.00,0.50,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.71,0.00,0.00


## Modelado de Tópicos

In [33]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

import pyLDAvis.lda_model

data = {
    'content': [
        'Me siento muy ansioso últimamente',
        'No puedo dormir debido a la ansiedad',
        'Me siento deprimido y sin energía',
        'La depresión está arruinando mi vida',
        'Estoy teniendo ataques de pánico frecuentemente',
        'La tristeza es constante y no sé qué hacer',
        'Tengo miedo de salir de casa por la ansiedad',
        'Mi ansiedad empeora cuando estoy en público',
        'No puedo concentrarme en el trabajo por la depresión',
        'Me siento solo y triste todo el tiempo',
        'La ansiedad no me deja disfrutar de la vida',
        'La depresión me hace sentir inútil',
        'Me siento atrapado en un ciclo de pensamientos negativos',
        'La ansiedad me causa problemas físicos como dolores de cabeza',
        'La depresión me impide hacer cosas que antes disfrutaba',
        'Siento un peso en el pecho por la ansiedad',
        'No tengo ganas de hacer nada debido a la depresión',
        'Mi ansiedad me hace evitar situaciones sociales',
        'La depresión me ha hecho perder el interés en mis hobbies',
        'Tengo problemas para respirar cuando tengo ansiedad',
        'La depresión me hace sentir que nada vale la pena',
        'Me preocupa constantemente lo que los demás piensan de mí',
        'No puedo dormir bien por las noches debido a la ansiedad',
        'La depresión me hace sentir aislado de todos',
        'Estoy agotado física y emocionalmente por la ansiedad',
        'No puedo disfrutar de las cosas debido a la depresión',
        'Siento que la ansiedad me está consumiendo',
        'La depresión me hace sentir que no tengo futuro',
        'Tengo pensamientos intrusivos debido a la ansiedad',
        'La depresión me hace sentir que no tengo propósito',
        'Siento una constante sensación de miedo por la ansiedad',
        'La depresión me hace sentir que estoy fallando en todo',
        'Me preocupo excesivamente por cosas pequeñas debido a la ansiedad',
        'La depresión me hace sentir que estoy atrapado',
        'Tengo ataques de ansiedad en situaciones inesperadas',
        'La depresión me hace sentir que estoy perdiendo el control de mi vida'
    ],
    'subreddit': [
        'Ansiedad', 'Ansiedad', 'Depresión', 'Depresión', 'Ansiedad', 'Depresión',
        'Ansiedad', 'Ansiedad', 'Depresión', 'Depresión', 'Ansiedad', 'Depresión',
        'Depresión', 'Ansiedad', 'Depresión', 'Ansiedad', 'Depresión', 'Ansiedad',
        'Depresión', 'Ansiedad', 'Depresión', 'Ansiedad', 'Depresión', 'Ansiedad',
        'Ansiedad', 'Depresión', 'Ansiedad', 'Depresión', 'Ansiedad', 'Depresión',
        'Ansiedad', 'Depresión', 'Ansiedad', 'Depresión', 'Ansiedad', 'Depresión'
    ]
}

df_example = pd.DataFrame(data)

vectorizer = CountVectorizer(max_features=1000, lowercase=True)
X = vectorizer.fit_transform(df_example['content'])


num_topics = 5 # select number of topics
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)


df_example['topic_probabilities'] = lda.transform(X).tolist()

In [34]:
for idx, topic in enumerate(lda_model.components_):
    print(f"Topic {idx+1}: {[vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-5:]]}")

Topic 1: ['agotado', 'atrapado', 'cabeza', 'ciclo', 'antes']
Topic 2: ['ataques', 'bien', 'causa', 'como', 'concentrarme']
Topic 3: ['ataques', 'aislado', 'ansiedad', 'casa', 'constante']


In [38]:
df_example.head(10)

,content,subreddit,topic_probabilities
0,Me siento muy ansioso últimamente,Ansiedad,"[0.03356967789015645, 0.033740931727065056, 0...."
1,No puedo dormir debido a la ansiedad,Ansiedad,"[0.028799025595053974, 0.02888614554644247, 0...."
2,Me siento deprimido y sin energía,Depresión,"[0.03356967789095624, 0.03374093172790791, 0.0..."
3,La depresión está arruinando mi vida,Depresión,"[0.028916374138421238, 0.028713632212852308, 0..."
4,Estoy teniendo ataques de pánico frecuentemente,Ansiedad,"[0.02911796736794761, 0.028943280985469614, 0...."
5,La tristeza es constante y no sé qué hacer,Depresión,"[0.9087409148067441, 0.02267218787410835, 0.02..."
6,Tengo miedo de salir de casa por la ansiedad,Ansiedad,"[0.02028758085025232, 0.9186782525443058, 0.02..."
7,Mi ansiedad empeora cuando estoy en público,Ansiedad,"[0.8991547657854091, 0.025145716025952773, 0.0..."
8,No puedo concentrarme en el trabajo por la dep...,Depresión,"[0.02029373187543719, 0.917741543710341, 0.021..."
9,Me siento solo y triste todo el tiempo,Depresión,"[0.02518549182649003, 0.025340744771584323, 0...."


In [36]:
pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "prueba.html")

c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


# Consignas

1) Terminar limpieza del DF: Eliminar duplicados, entradas en otro idioma (langdetect), nulos, etc

2) Trabajar la columna de contenido y exponer las diferencias entre stemming y lematización a partir de una visualización de los términos más frecuentes de cada subreddit luego de aplicar estas técnicas. Seleccionar la que consideren más adecuada e interpretar los resultados (pueden utilizar un gráfico de barras o una nube de palabras).

3) Con el df limpio volver a trabajar la variable fecha, elegir una visualización que les resulte interesante e **interpretar** los resultados.

4) Implementar una técnica de vectorización (BoW o TF-IDF)

5) Realizar un modelado de tópicos con al menos dos números de clusters diferentes, comentar los resultados de cada intento. ¿Alguno de los dos modelos les parece que ofrece resultados más coherentes? ¿Pueden a partir de los términos más relevantes para cada tópico imaginar el tema que los nuclea?